In [4]:
import pandas as pd

In [5]:
df=pd.read_csv('chorales/train/chorale_000.csv')

In [6]:
df

,note0,note1,note2,note3
0,74,70,65,58
1,74,70,65,58
2,74,70,65,58
3,74,70,65,58
4,75,70,58,55
...,...,...,...,...
187,70,65,62,46
188,70,65,62,46
189,70,65,62,46
190,70,65,62,46


In [12]:
import os

train_files = sorted([os.path.join('chorales', 'train', f) for f in os.listdir(os.path.join('chorales', 'train')) if f.endswith('.csv')])
valid_files = sorted([os.path.join('chorales', 'valid', f) for f in os.listdir(os.path.join('chorales', 'valid')) if f.endswith('.csv')])
test_files = sorted([os.path.join('chorales', 'test', f) for f in os.listdir(os.path.join('chorales', 'test')) if f.endswith('.csv')])

In [ ]:
train_data = [pd.read_csv(f).values.tolist() for f in train_files]
valid_data = [pd.read_csv(f).values.tolist() for f in valid_files]
test_data = [pd.read_csv(f).values.tolist() for f in test_files]

- 31 => C1
- 81 => A5
- 0 => SILENCE

In [14]:
train_data[0]

[[74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [75, 70, 58, 55],
 [75, 70, 58, 55],
 [75, 70, 60, 55],
 [75, 70, 60, 55],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 70, 62, 55],
 [77, 70, 62, 55],
 [77, 69, 62, 55],
 [77, 69, 62, 55],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [77, 65, 62, 50],
 [77, 65, 62, 50],
 [77, 65, 60, 50],
 [77, 65, 60, 50],
 [74, 67, 58, 55],
 [74, 67, 58, 55],
 [74, 67, 58, 53],
 [74, 67, 58, 53],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 65, 57

In [24]:
from music21 import stream, chord

chorale = train_data[20]
s = stream.Stream()
for row in chorale:
    s.append(chord.Chord([n for n in row if n], quarterLength=1))
s.show('midi')

### Preprocessing data

In [16]:
import numpy as np

min_note, max_note = 36, 81
window_size, window_offset, batch_size = 32, 16, 32


def make_xy(chorales):
    # get segments of 33 chords with 16 chords offset between them
    windows = [c[i:i + window_size + 1] for c in chorales for i in range(0, len(c) - window_size, window_offset)]

    data = np.array(windows, dtype=int)
    # if note is 0, keep it, otherwise rescale notes from 36-81 to 1-46
    data = np.where(data==0, 0, data - min_note + 1)
    # make the range 0-46 in total
    data = np.clip(data, 0, max_note - min_note + 1)

    flat = data.reshape(data.shape[0], -1)

    # return every chord except the last one for X (32 notes) and everything except the first one for Y (32 notes)
    return flat[:, :-1], flat[:, 1:]


X_train, Y_train = make_xy(train_data)
X_valid, Y_valid = make_xy(valid_data)
X_test,  Y_test  = make_xy(test_data)

In [17]:
X_train.shape

(3111, 131)

### Training model

In [30]:
from tensorflow.keras.models import Sequential

ImportError: Traceback (most recent call last):
  File "c:\Users\Len\Desktop\bach chorals music\.venv\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.